In [1]:
import numpy as np
import pyvista as pv

In [9]:
def cells_to_cell_list(cells):
    cells = cells.copy()
    cell_list = list()
    while len(cells):
        cell_len = cells[0]
        cell = cells[1:cell_len + 1]
        cell_list.append(cell)
        cells = cells[cell_len+1:]
    return cell_list

def cell_list_to_cells(cell_list):
    cells = list()
    for cell in cell_list:
        cell_len = len(cell)
        cells.append(cell_len)
        cells += list(cell)
    return np.array(cells)

def delete_degenerate_cells(mesh):
    points = mesh.points
    cells = mesh.cells
    celltypes = mesh.celltypes
    cell_list = cells_to_cell_list(cells)
    
    cleaned_cell_list = list()
    cleaned_celltypes = list()
    for cell, celltype in zip(*(cell_list, celltypes)):
        if len(cell) == len(set(cell)):
            cleaned_cell_list.append(cell)
            cleaned_celltypes.append(celltype)
    cleaned_cells = cell_list_to_cells(cleaned_cell_list)
    cleaned_celltypes = np.array(cleaned_celltypes)

    fixed_mesh = pv.UnstructuredGrid(cleaned_cells, cleaned_celltypes, points)
    return fixed_mesh


In [10]:
r_1d = np.linspace(0, 1, 10)
theta_1d = np.linspace(0, np.pi, 10)
phi_1d = np.linspace(0, 2*np.pi, 10)

r, theta, phi = np.meshgrid(r_1d, theta_1d, phi_1d, indexing='ij')

x = r * np.sin(theta) * np.cos(phi)
y = r * np.sin(theta) * np.sin(phi)
z = r * np.cos(theta)

mesh = pv.StructuredGrid(x, y, z)
unstructured_mesh = mesh.cast_to_unstructured_grid()
cleaned_mesh = unstructured_mesh.clean(tolerance=1e-4)
tri_mesh = cleaned_mesh.triangulate()
fixed_mesh = delete_degenerate_cells(tri_mesh)

In [11]:
def get_degenerate_cell_indices(mesh):
    cells = mesh.cells.copy()
    cell_list = cells_to_cell_list(cells)
    degenerate_cell_indices = list()
    for cell_idx, cell in enumerate(cell_list):
        if len(cell) != len(set(cell)):
            degenerate_cell_indices.append(cell_idx)
    return degenerate_cell_indices

def plot_wireframe_with_degenerate_cells(mesh):
    degenerate_cell_indices = get_degenerate_cell_indices(mesh)

    pl = pv.Plotter(notebook=False)
    pl.add_mesh(mesh.points, color='black', opacity=0.2)
    pl.add_mesh(mesh.extract_all_edges(), color='blue', opacity=0.2)
    if len(degenerate_cell_indices) > 0:
        pl.add_mesh(mesh.extract_cells(degenerate_cell_indices).extract_all_edges(), color='red', opacity=1)
    else:
        print('No degenerate cells')
    pl.show()

In [14]:
plot_wireframe_with_degenerate_cells(tri_mesh)

In [15]:
plot_wireframe_with_degenerate_cells(fixed_mesh)

No degenerate cells


In [20]:
import numpy as np
import pyvista as pv

nr=5
ntheta=5
nphi=8


def spherical_to_cartesian(r, theta, phi):
    # takes in 1D ndarrays or floats for each, returns (n,3) points in cartesian
    r, theta, phi = np.meshgrid(r, theta, phi, indexing='ij')

    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    return np.vstack((x.ravel(), y.ravel(), z.ravel())).transpose()

#r_1d = np.insert(np.geomspace(0.1, 10, nr-1), 0, 0.0)
r_1d = np.linspace(0, 10, nr)
theta_1d = np.linspace(0, np.pi, ntheta)
phi_1d = np.linspace(0, 2*np.pi, nphi)

#points on axis
points = [[0., 0., 0.]]
points.extend(spherical_to_cartesian(r_1d[1:], theta_1d[0], phi_1d[0]))
points.extend(spherical_to_cartesian(r_1d[1:], theta_1d[-1], phi_1d[0]))

# rest of points with phi changing quickest
for ir in r_1d[1:]:
    for itheta in theta_1d[1:-1]:
        points.extend(spherical_to_cartesian(ir, itheta, phi_1d[:-1]))

cells = []
celltypes = []

 # 2*nr-1 points on axis
 # At each r, theta, nphi-1 points
 # At each r, ntheta-2 rings of points: theta[0] and theta[-1] are on the axis

# first make the tetras that form with origin and axis point
#   origin is 0
#   first axis point is 1
#   other points at 2*nr-1 + (0:nphi-1)

for iphi in range(nphi-1):
    start = 2*nr-1
    upper_range = (1 + iphi) % (nphi - 1)
    cells.append(4)
    cells.extend([0, 1, start+iphi, start+upper_range])

    celltypes.append(pv.CellType.TETRA)

# next tetras that form with origin and bottom axis point
# 2 theta points are already on axis and we want the last remaning theta level, so (ntheta-3)
# other points at 2*nr-1 + (ntheta-3)*(nphi-1) + (0:nphi-1)

for iphi in range(nphi-1):
    start = 2*nr - 1 + (ntheta - 3) * (nphi - 1)
    upper_range = (1 + iphi) % (nphi - 1)
    cells.append(4)
    cells.extend([0, nr, start+iphi, start+upper_range])

    celltypes.append(pv.CellType.TETRA)

# Pyramids that form to origin but without an axis point
# start with the same points as above, but connect to next theta points

for itheta in range(ntheta-3):
    for iphi in range(nphi-1):
        start = 2*nr - 1 + itheta*(nphi-1)
        upper_range = (1 + iphi) % (nphi - 1)
        next = 2*nr - 1 + (itheta+1)*(nphi-1)
        cells.append(5)
        cells.extend([start+iphi, start+upper_range, next+upper_range, next+iphi, 0])

        celltypes.append(pv.CellType.PYRAMID)

# Wedges form between two r levels at first and last theta position
#   At each r level, the triangle is formed with axis point,  two phi positions
# First go upwards
for ir in range(nr-2):
    axis0 = ir + 1
    axis1 = ir + 2
    for iphi in range(nphi-1):
        start = 2*nr-1 + ir*(nphi-1)*(ntheta-2)
        upper_range = (1 + iphi) % (nphi - 1)

        next = 2*nr-1 + (ir+1)*(nphi-1)*(ntheta-2)
        cells.append(6)
        cells.extend([axis0, start+iphi, start+upper_range, axis1, next+iphi, next+upper_range])

        celltypes.append(pv.CellType.WEDGE) 

# now go downwards
for ir in range(nr-2):
    axis0 = nr + ir
    axis1 = nr + ir + 1
    for iphi in range(nphi-1):
        start = 2*nr-1 + ir*(nphi-1)*(ntheta-2) + (ntheta - 3) * (nphi - 1)
        upper_range = (1 + iphi) % (nphi - 1)

        next = 2*nr-1 + (ir+1)*(nphi-1)*(ntheta-2) + (ntheta - 3) * (nphi - 1)
        cells.append(6)
        cells.extend([axis0, start+iphi, start+upper_range, axis1, next+iphi, next+upper_range])

        celltypes.append(pv.CellType.WEDGE) 


# Form Hexahedra
# Hexahedra form between two r levels and two theta levels and two phi levels
#   Order by r levels
for ir in range(nr-2):
    for itheta in range(ntheta-3):
        for iphi in range(nphi-1):
            start0 = 2*nr-1 + ir*(nphi-1)*(ntheta-2) + itheta*(nphi - 1)
            upper_range = (1 + iphi) % (nphi - 1)

            next0 = 2*nr-1 + ir*(nphi-1)*(ntheta-2)  + (itheta + 1)*(nphi - 1)
            start1 = 2*nr-1 + (ir+1)*(nphi-1)*(ntheta-2) + itheta*(nphi - 1)
            next1 = 2*nr-1 + (ir+1)*(nphi-1)*(ntheta-2)  + (itheta + 1)*(nphi - 1)


            cells.append(8)
            cells.extend([start0+iphi, start0+upper_range, next0+upper_range, next0+iphi, start1+iphi, start1+upper_range, next1+upper_range, next1+iphi])

            celltypes.append(pv.CellType.HEXAHEDRON) 

mesh = pv.UnstructuredGrid(cells, celltypes, points)

mesh_cell_center = mesh.cell_centers()
mesh["radial_position"] = np.linalg.norm(mesh_cell_center.points, axis=1)

pl = pv.Plotter()
pl.add_mesh(mesh.explode(1.25))
pl.show()

Widget(value="<iframe src='http://localhost:58422/index.html?ui=P_0x1dadd5c27f0_0&reconnect=auto' style='width…